In [1]:
import os, sys
sys.path += [os.path.dirname('../scripts/')]

In [3]:
model_ckpt = 'CL_valaro_z64_bs512'

In [4]:
from transformers import ViTFeatureExtractor, ViTForImageClassification

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

/home/rkn/.local/share/virtualenvs/AffectNet-experiments-e17pgDV_/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
from dataset import AlternatingDataset, AffectNetDatasetForSupConWithValenceArousal
from torchaffectnet import AffectNetDatasetForSupCon
from torchvision.transforms import (Compose,
                                    Normalize,
                                    Resize,
                                    RandomResizedCrop,
                                    RandomHorizontalFlip,
                                    RandomApply,
                                    ColorJitter,
                                    RandomGrayscale,
                                    ToTensor,
                                    RandomAffine)

normalize = Normalize(mean=feature_extractor.image_mean,
                          std=feature_extractor.image_std)

transform1 = Compose([
    RandomAffine(30),
    Resize(tuple(feature_extractor.size.values())),
    ToTensor(),
    normalize,
])

transform2 = Compose([
    RandomResizedCrop(size=tuple(
        feature_extractor.size.values()), scale=(0.2, 1.)),
    RandomHorizontalFlip(),
    RandomApply([
        ColorJitter(0.4, 0.4, 0.4, 0.1)
    ], p=0.8),
    ToTensor(),
    normalize
])

valaro_dataset = AffectNetDatasetForSupConWithValenceArousal('../../Affectnet/validation.csv',
                                                             '../../Affectnet/Manually_Annotated/Manually_Annotated_Images/',
                                                             transform1=transform1,
                                                             transform2=transform2,
                                                             exclude_label=[8,9,10])
expression_dataset = AffectNetDatasetForSupCon('../../Affectnet/validation.csv',
                                               '../../Affectnet/Manually_Annotated/Manually_Annotated_Images/',
                                               transform1=transform1,
                                               transform2=transform2,
                                               exclude_label=[8,9,10])

dataset = AlternatingDataset(valaro_dataset, expression_dataset, batch_size=4, alter_steps=2)
dataset

In [5]:
from torch.utils.data import DataLoader
from dataset import AlternatingContrastiveCollator

train_dataloader = DataLoader(dataset, collate_fn=AlternatingContrastiveCollator(), batch_size=4)
iter_ = iter(train_dataloader)

In [6]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4, 2])
0


In [7]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4, 2])
0


In [8]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4])
1


In [9]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4])
1


In [10]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4, 2])
0


In [11]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4, 2])
0


In [12]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4])
1


In [13]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4])
1


In [14]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4, 2])
0


In [15]:
batch = next(iter_)
print(batch['pixel_values'].shape)
print(batch['labels'].shape)
print(batch['dataset_id'])

torch.Size([8, 3, 224, 224])
torch.Size([4, 2])
0
